In [45]:
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.datasets import make_classification
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from transformers import pipeline
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense

def model_dnn(look_back):
    model=Sequential()
    model.add(Dense(units=32, input_dim=look_back, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',  optimizer='adam',metrics = ['mape', 'mae'])
    return model

def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def convert2matrix(data_arr, look_back):
 X, Y =[], []
 for i in range(len(data_arr)-look_back):
  d=i+look_back  
  X.append(data_arr[i:d,0])
  Y.append(data_arr[d,0])
 return np.array(X), np.array(Y)

df = pd.read_csv('test.csv')
df.drop(['Trades', 'Deliverable Volume', 'Volume', 'Turnover', '%Deliverble', 'Symbol', 'Series'], inplace = True,axis=1)

y = pd.DataFrame({'Date': df['Date'], 'Open': df['Open']})
y['Date'] = pd.to_datetime(df['Date'])
y.set_index('Date', inplace=True)
y = y.resample('D').asfreq()
y['Open'] = y['Open'].ffill()

In [62]:
train_size =4413
train, test =y.values[0:train_size,:],y.values[train_size:len(y.values),:]

look_back = 30

trainX, trainY = convert2matrix(train, look_back)
testX, testY = convert2matrix(test, look_back)

In [63]:
model=model_dnn(look_back)
history=model.fit(trainX,trainY, epochs=100, batch_size=30, verbose=2, validation_data= (testX,testY),shuffle=False)

Epoch 1/100
147/147 - 0s - loss: 20286.2227 - mape: 10.1032 - mae: 46.3813 - val_loss: 1017.1178 - val_mape: 5.6733 - val_mae: 23.9485 - 241ms/epoch - 2ms/step
Epoch 2/100
147/147 - 0s - loss: 1725.5858 - mape: 6.2000 - mae: 19.7129 - val_loss: 941.2574 - val_mape: 5.3435 - val_mae: 22.9482 - 70ms/epoch - 476us/step
Epoch 3/100
147/147 - 0s - loss: 1399.2900 - mape: 6.4747 - mae: 18.9950 - val_loss: 854.7000 - val_mape: 4.9998 - val_mae: 21.7304 - 69ms/epoch - 467us/step
Epoch 4/100
147/147 - 0s - loss: 1185.3168 - mape: 6.5622 - mae: 18.3087 - val_loss: 642.8813 - val_mape: 4.2161 - val_mae: 18.5015 - 68ms/epoch - 463us/step
Epoch 5/100
147/147 - 0s - loss: 1049.9165 - mape: 5.7933 - mae: 16.6721 - val_loss: 668.2249 - val_mape: 4.2874 - val_mae: 18.8944 - 68ms/epoch - 460us/step
Epoch 6/100
147/147 - 0s - loss: 997.9302 - mape: 5.7792 - mae: 16.4092 - val_loss: 550.7930 - val_mape: 3.8351 - val_mae: 16.7660 - 70ms/epoch - 474us/step
Epoch 7/100
147/147 - 0s - loss: 967.5874 - mape: 5

147/147 - 0s - loss: 329.8709 - mape: 2.6317 - mae: 8.2286 - val_loss: 205.4864 - val_mape: 2.1206 - val_mae: 9.1605 - 68ms/epoch - 459us/step
Epoch 54/100
147/147 - 0s - loss: 330.1309 - mape: 2.8572 - mae: 8.4653 - val_loss: 202.8357 - val_mape: 2.0883 - val_mae: 9.0252 - 68ms/epoch - 462us/step
Epoch 55/100
147/147 - 0s - loss: 326.4737 - mape: 2.7902 - mae: 8.3382 - val_loss: 204.1540 - val_mape: 2.0980 - val_mae: 9.0806 - 66ms/epoch - 451us/step
Epoch 56/100
147/147 - 0s - loss: 322.6268 - mape: 2.7799 - mae: 8.2793 - val_loss: 201.4143 - val_mape: 2.0795 - val_mae: 8.9918 - 67ms/epoch - 457us/step
Epoch 57/100
147/147 - 0s - loss: 319.7152 - mape: 2.7742 - mae: 8.2454 - val_loss: 200.0370 - val_mape: 2.0705 - val_mae: 8.9498 - 67ms/epoch - 459us/step
Epoch 58/100
147/147 - 0s - loss: 314.2371 - mape: 2.7206 - mae: 8.1165 - val_loss: 198.9648 - val_mape: 2.0617 - val_mae: 8.9145 - 67ms/epoch - 456us/step
Epoch 59/100
147/147 - 0s - loss: 314.7877 - mape: 2.7813 - mae: 8.1974 - val

In [64]:
train_score = model.evaluate(trainX, trainY, verbose=0)
print('Train Mean Absolute Percentile Error: %.2f; Train Mean Absolute Error(MAE) : %.2f ' 
% (np.sqrt(train_score[1]), train_score[2]))
test_score = model.evaluate(testX, testY, verbose=0)
print('Test Mean Absolute Percentile Error: %.2f; Test Mean Absolute Error(MAE) : %.2f ' 
% (np.sqrt(test_score[1]), test_score[2]))


Train Mean Absolute Percentile Error: 1.51; Train Mean Absolute Error(MAE) : 6.97 
Test Mean Absolute Percentile Error: 1.40; Test Mean Absolute Error(MAE) : 8.53 
